In [278]:
import pandas as pd
import numpy as np
import random

In [303]:
df_train = pd.read_parquet(path="./ebnerd_small/train/behaviors.parquet")
df_train = df_train[["impression_id", "user_id", "article_ids_inview", "article_ids_clicked"]]

In [310]:
df_val = pd.read_parquet(path="./ebnerd_small/validation/behaviors.parquet")
df_val = df_val[["impression_id", "user_id", "article_ids_inview", "article_ids_clicked"]]
df_val

,impression_id,user_id,article_ids_inview,article_ids_clicked
0,96791,22548,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696]
1,96798,22548,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281]
2,96801,22548,"[9784648, 7184889, 9784444, 9781785, 9783405, ...",[9784444]
3,96808,22548,"[9784607, 9695098, 9784406, 7184889, 9781983]",[9781983]
4,96810,22548,"[9781983, 7184889, 9784642, 9782108, 9781785, ...",[9784642]
...,...,...,...,...
244642,579250707,1529985,"[9782929, 9782919, 9783213, 9783075, 9782746, ...",[9783075]
244643,579251515,1532837,"[9782746, 9782929, 9783720, 9782645, 9783075]",[9783075]
244644,579251519,1532837,"[9783213, 9782645, 9782423, 9783657, 9782746, ...",[9782423]
244645,579253363,1539939,"[9142564, 9783657, 9783404, 9782670, 9783720, ...",[9783720]


In [82]:
# Random baseline
guessed_right = 0
guessed_wrong = 0
for index, row in df_val.iterrows():
    if random.choice(row["article_ids_inview"]) in row["article_ids_clicked"]:
        guessed_right += 1
    else:
        guessed_wrong += 1
print(guessed_right, guessed_wrong)
guessed_right/guessed_wrong

28812 215835


In [304]:
# Construction of all guesses
df_answers = df_train[['impression_id', "user_id", "article_ids_inview", "article_ids_clicked"]].copy()
df_answers

,impression_id,user_id,article_ids_inview,article_ids_clicked
0,149474,139836,"[9778623, 9778682, 9778669, 9778657, 9778736, ...",[9778657]
1,150528,143471,"[9778718, 9778728, 9778745, 9778669, 9778657, ...",[9778623]
2,153068,151570,"[9778657, 9778669, 9772866, 9776259, 9756397, ...",[9778669]
3,153070,151570,"[9020783, 9778444, 9525589, 7213923, 9777397, ...",[9778628]
4,153071,151570,"[9777492, 9774568, 9565836, 9335113, 9771223, ...",[9777492]
...,...,...,...,...
232882,580099643,2106715,"[9233208, 9771242, 9767697, 9514481, 9771065, ...",[9770886]
232883,580099644,2106715,"[9771065, 9767697, 9770886, 9758882, 9709817, ...",[9769306]
232884,580099645,2106715,"[9771042, 9440508, 9486080, 9770997, 9120051, ...",[9771042]
232885,580100695,2110744,"[9440508, 9142581, 9769917, 9767697, 9514481, ...",[9767697]


In [254]:
guessed_right = 0
guessed_wrong = 0
guesses = []
for index, row in df_train.iterrows():
    curUser = row["user_id"]
    allClicked = np.empty([0,0]) if len(df_train[df_train["user_id"] == curUser]["article_ids_clicked"].values) == 0 else np.concatenate(df_train[df_train["user_id"] == curUser]["article_ids_clicked"].values)
    leftToChooseFrom = [i for i in row["article_ids_inview"] if i not in allClicked]
    if len(leftToChooseFrom) == 0:
        guess = random.choice(row["article_ids_inview"])
        guesses.append([])
    else:
        guesses.append(leftToChooseFrom)
        guess = random.choice(leftToChooseFrom)
    if guess in row["article_ids_clicked"]:
        guessed_right += 1
    else:
        guessed_wrong += 1
print(guessed_right, guessed_wrong)
print(guessed_right/guessed_wrong)
print(guessed_right/(guessed_wrong + guessed_right))
df_answers["not_already_clicked"] = guesses
df_answers

28983 215664
0.13438960605386155
0.11846865074985591


,impression_id,user_id,article_ids_inview,article_ids_clicked,not_already_liked
0,96791,22548,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],"[9783865, 9784591, 9784679, 9784696, 9784710]"
1,96798,22548,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],"[9782884, 9783865, 9782726, 9695098, 9782806, ..."
2,96801,22548,"[9784648, 7184889, 9784444, 9781785, 9783405, ...",[9784444],"[9784648, 7184889, 9784444, 9781785, 9783405, ..."
3,96808,22548,"[9784607, 9695098, 9784406, 7184889, 9781983]",[9781983],"[9784607, 9695098, 9784406, 7184889, 9781983]"
4,96810,22548,"[9781983, 7184889, 9784642, 9782108, 9781785, ...",[9784642],"[9781983, 7184889, 9784642, 9782108, 9781785, ..."
...,...,...,...,...,...
244642,579250707,1529985,"[9782929, 9782919, 9783213, 9783075, 9782746, ...",[9783075],"[9782929, 9782919, 9783213, 9783075, 9782746, ..."
244643,579251515,1532837,"[9782746, 9782929, 9783720, 9782645, 9783075]",[9783075],"[9782746, 9782929, 9783720, 9782645, 9783075]"
244644,579251519,1532837,"[9783213, 9782645, 9782423, 9783657, 9782746, ...",[9782423],"[9783213, 9782645, 9782423, 9783657, 9782746, ..."
244645,579253363,1539939,"[9142564, 9783657, 9783404, 9782670, 9783720, ...",[9783720],"[9142564, 9783657, 9783404, 9782670, 9783720, ..."


In [256]:
df_answers

,impression_id,user_id,article_ids_inview,article_ids_clicked,not_already_clicked
0,96791,22548,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],"[9783865, 9784591, 9784679, 9784696, 9784710]"
1,96798,22548,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],"[9782884, 9783865, 9782726, 9695098, 9782806, ..."
2,96801,22548,"[9784648, 7184889, 9784444, 9781785, 9783405, ...",[9784444],"[9784648, 7184889, 9784444, 9781785, 9783405, ..."
3,96808,22548,"[9784607, 9695098, 9784406, 7184889, 9781983]",[9781983],"[9784607, 9695098, 9784406, 7184889, 9781983]"
4,96810,22548,"[9781983, 7184889, 9784642, 9782108, 9781785, ...",[9784642],"[9781983, 7184889, 9784642, 9782108, 9781785, ..."
...,...,...,...,...,...
244642,579250707,1529985,"[9782929, 9782919, 9783213, 9783075, 9782746, ...",[9783075],"[9782929, 9782919, 9783213, 9783075, 9782746, ..."
244643,579251515,1532837,"[9782746, 9782929, 9783720, 9782645, 9783075]",[9783075],"[9782746, 9782929, 9783720, 9782645, 9783075]"
244644,579251519,1532837,"[9783213, 9782645, 9782423, 9783657, 9782746, ...",[9782423],"[9783213, 9782645, 9782423, 9783657, 9782746, ..."
244645,579253363,1539939,"[9142564, 9783657, 9783404, 9782670, 9783720, ...",[9783720],"[9142564, 9783657, 9783404, 9782670, 9783720, ..."


In [223]:
df_articles_train = pd.read_parquet(path="./ebnerd_small/articles.parquet")
# df_articles_val = pd.read_parquet(path="./ebnerd_small/articles.parquet")
# df_articles_train = df_articles_train[["article_id", "published_time", "last_modified_time", "premium", "category"]]
df_articles_train

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3001353,Natascha var ikke den første,"Politiet frygter nu, at Nataschas bortfører ha...",2023-06-29 06:20:33,False,Sagen om den østriske Natascha og hendes bortf...,2006-08-31 08:06:45,[3150850],article_default,https://ekstrabladet.dk/krimi/article3001353.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9955,Negative
1,3003065,Kun Star Wars tjente mere,Biografgængerne strømmer ind for at se 'Da Vin...,2023-06-29 06:20:35,False,Vatikanet har opfordret til at boykotte filmen...,2006-05-21 16:57:00,[3006712],article_default,https://ekstrabladet.dk/underholdning/filmogtv...,...,[],"[Underholdning, Film og tv, Økonomi]",414,"[433, 434]",underholdning,NaN,NaN,NaN,0.8460,Positive
2,3012771,Morten Bruun fyret i SønderjyskE,FODBOLD: Morten Bruun fyret med øjeblikkelig v...,2023-06-29 06:20:39,False,Kemien mellem spillerne i Superligaklubben Søn...,2006-05-01 14:28:40,[3177953],article_default,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,...,[],"[Erhverv, Kendt, Sport, Fodbold, Ansættelsesfo...",142,"[196, 199]",sport,NaN,NaN,NaN,0.8241,Negative
3,3023463,Luderne flytter på landet,I landets tyndest befolkede områder skyder bor...,2023-06-29 06:20:43,False,Det frække erhverv rykker på landet. I den tyn...,2007-03-24 08:27:59,[3184029],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Livsstil, Erotik]",118,[133],nyheder,NaN,NaN,NaN,0.7053,Neutral
4,3032577,Cybersex: Hvornår er man utro?,En flirtende sms til den flotte fyr i regnskab...,2023-06-29 06:20:46,False,"De fleste af os mener, at et tungekys er utros...",2007-01-18 10:30:37,[3030463],article_default,https://ekstrabladet.dk/sex_og_samliv/article3...,...,[],"[Livsstil, Partnerskab]",565,[],sex_og_samliv,NaN,NaN,NaN,0.9307,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20733,9803492,Vilde billeder: Vulkan i udbrud i ferieparadis,Der er gang i vulkanen på Hawaiis største ø,2023-06-29 06:49:26,False,Det spyer med lava fra vulkanen Kilauea på Haw...,2023-06-08 05:49:20,"[9803493, 9803494, 9803495, 9803495, 9803494]",article_default,https://ekstrabladet.dk/nyheder/samfund/vilde-...,...,"[LOC, LOC, PER, ORG, ORG]","[Katastrofe, Vejr, Større katastrofe]",118,[133],nyheder,535989.0,100120.0,4112624.0,0.6095,Neutral
20734,9803505,Flyvende Antonsen knuser topspiller,"Verdens nummer syv, Chou Tien-Chen, fik ikke e...",2023-06-29 06:49:26,False,Anders Antonsen har holdt pause fra badmintonb...,2023-06-08 05:54:06,[9803516],article_default,https://ekstrabladet.dk/sport/anden_sport/badm...,...,"[PER, PROD, PER, PER, MISC, MISC, PER, PER, LO...","[Kendt, Begivenhed, Sport, Ketcher- og batspor...",142,"[327, 330]",sport,13320.0,959.0,55691.0,0.8884,Positive
20735,9803525,"Dansk skuespiller: - Jeg nægtede, at jeg var syg",Julie R. Ølgaard fik akut kejsersnit og fødte ...,2023-06-29 06:49:26,False,"Mens hun lå søvnløs, lød kakofonien fra baggår...",2023-06-08 06:45:46,"[9803518, 9803519, 9803520, 9803521, 9803522, ...",article_default,https://ekstrabladet.dk/underholdning/dkkendte...,...,"[PER, PROD, PER, PER, PER, PER, MISC]","[Kendt, Livsstil, Familieliv, Underholdning, F...",414,[425],underholdning,315391.0,50361.0,2550671.0,0.7737,Negative
20736,9803560,Så slemt er det: 14.000 huse er oversvømmet,Tusindvis af huse står under vand i Kherson-re...,2023-06-29 06:49:26,False,Et område på omkring 600 kvadratkilometer står...,2023-06-08 06:25:42,None,article_default,https://ekstrabladet.dk/nyheder/saa-slemt-er-d...,...,"[LOC, LOC, LOC, PROD, PER, LOC, ORG, ORG, LOC]","[International politik, Katastrofe, Større kat...",118,[],nyheder,21318.0,1237.0,67514.0,0.9927,Negative


                  19379   19420   19427   19428   19430
sentiment_score  0.9793     NaN     NaN     NaN     NaN
sentiment_score     NaN  0.9823     NaN     NaN     NaN
sentiment_score     NaN     NaN  0.9781     NaN     NaN
sentiment_score     NaN     NaN     NaN  0.9529     NaN
sentiment_score     NaN     NaN     NaN     NaN  0.8887


In [190]:
# sort them by sentiment_score

In [305]:
guessed_right = 0
guessed_wrong = 0
guesses = []
for index, row in df_train.iterrows():
    a = df_articles_train[df_articles_train["article_id"].isin(row["article_ids_inview"])]
    sorted_art = a[["article_id", "sentiment_score"]].sort_values(by=['sentiment_score'], ascending=False)
    guesses.append(sorted_art["article_id"].tolist())
    if sorted_art.loc[sorted_art.index[0]]["article_id"].astype(int) in row["article_ids_clicked"]:
        guessed_right += 1
    else:
        guessed_wrong += 1

print(guessed_right, guessed_wrong)
print(guessed_right/guessed_wrong)
print(guessed_right/(guessed_wrong + guessed_right))
df_answers["highest_sentiment_score"] = guesses
df_answers

29887 203000
0.14722660098522167
0.1283326248352205


,impression_id,user_id,article_ids_inview,article_ids_clicked,highest_sentiment_score
0,149474,139836,"[9778623, 9778682, 9778669, 9778657, 9778736, ...",[9778657],"[9778623, 9778728, 9778682, 9778669, 9778657, ..."
1,150528,143471,"[9778718, 9778728, 9778745, 9778669, 9778657, ...",[9778623],"[9778718, 9778623, 9778728, 9778682, 9778669, ..."
2,153068,151570,"[9778657, 9778669, 9772866, 9776259, 9756397, ...",[9778669],"[9776259, 9778682, 9778669, 9756397, 9778657, ..."
3,153070,151570,"[9020783, 9778444, 9525589, 7213923, 9777397, ...",[9778628],"[9778444, 9430567, 9778718, 9777397, 9525589, ..."
4,153071,151570,"[9777492, 9774568, 9565836, 9335113, 9771223, ...",[9777492],"[9770218, 9778623, 9777492, 9131971, 9335113, ..."
...,...,...,...,...,...
232882,580099643,2106715,"[9233208, 9771242, 9767697, 9514481, 9771065, ...",[9770886],"[9770886, 9771242, 9771083, 9769306, 9767697, ..."
232883,580099644,2106715,"[9771065, 9767697, 9770886, 9758882, 9709817, ...",[9769306],"[9770886, 9742735, 9771242, 9709817, 9769917, ..."
232884,580099645,2106715,"[9771042, 9440508, 9486080, 9770997, 9120051, ...",[9771042],"[9770886, 9767697, 9440508, 9470078, 9770997, ..."
232885,580100695,2110744,"[9440508, 9142581, 9769917, 9767697, 9514481, ...",[9767697],"[9770886, 9769917, 9767697, 9514481, 9440508, ..."


In [258]:
guessed_right = 0
guessed_wrong = 0
guesses = []
for index, row in df_val.iterrows():
    a = df_articles_train[df_articles_train["article_id"].isin(row["article_ids_inview"])]
    sorted_art = a[["article_id", "sentiment_score"]].sort_values(by=['sentiment_score'], ascending=True)
    guesses.append(sorted_art.loc[sorted_art.index[0]]["article_id"].astype(int))
    if sorted_art.loc[sorted_art.index[0]]["article_id"].astype(int) in row["article_ids_clicked"]:
        guessed_right += 1
    else:
        guessed_wrong += 1

print(guessed_right, guessed_wrong)
print(guessed_right/guessed_wrong)
print(guessed_right/(guessed_wrong + guessed_right))

df_answers["lowest_sentiment_score"] = guesses
df_answers

30966 213681
0.14491695564884102
0.12657420691853977


,impression_id,user_id,article_ids_inview,article_ids_clicked,not_already_clicked,highestsentiment_score,lowest_sentiment_score
0,96791,22548,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],"[9783865, 9784591, 9784679, 9784696, 9784710]",9784591,9784710
1,96798,22548,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],"[9782884, 9783865, 9782726, 9695098, 9782806, ...",9779370,9784506
2,96801,22548,"[9784648, 7184889, 9784444, 9781785, 9783405, ...",[9784444],"[9784648, 7184889, 9784444, 9781785, 9783405, ...",9782108,9784444
3,96808,22548,"[9784607, 9695098, 9784406, 7184889, 9781983]",[9781983],"[9784607, 9695098, 9784406, 7184889, 9781983]",7184889,9781983
4,96810,22548,"[9781983, 7184889, 9784642, 9782108, 9781785, ...",[9784642],"[9781983, 7184889, 9784642, 9782108, 9781785, ...",9782108,9781983
...,...,...,...,...,...,...,...
244642,579250707,1529985,"[9782929, 9782919, 9783213, 9783075, 9782746, ...",[9783075],"[9782929, 9782919, 9783213, 9783075, 9782746, ...",9783075,9783213
244643,579251515,1532837,"[9782746, 9782929, 9783720, 9782645, 9783075]",[9783075],"[9782746, 9782929, 9783720, 9782645, 9783075]",9783720,9782929
244644,579251519,1532837,"[9783213, 9782645, 9782423, 9783657, 9782746, ...",[9782423],"[9783213, 9782645, 9782423, 9783657, 9782746, ...",9783720,9783657
244645,579253363,1539939,"[9142564, 9783657, 9783404, 9782670, 9783720, ...",[9783720],"[9142564, 9783657, 9783404, 9782670, 9783720, ...",9783720,9782092


In [192]:
# category

In [219]:
unique_categories = df_articles_train["category"].unique()
unique_categories

array([ 140,  414,  142,  118,  565,    2,  539,  498,  572,  512,  457,
        529,  561,  806, 2341,  731,   22, 1505,   68, 2077, 2504, 2731,
       2737, 2889, 2975], dtype=int16)

In [211]:
def aggregate_hobbies(hobbies):
    aggregated = []
    for hobby_list in hobbies:
        aggregated.extend(hobby_list)
    return aggregated

df_history_train = pd.read_parquet(path="./ebnerd_small/train/history.parquet")
df_history_val = pd.read_parquet(path="./ebnerd_small/validation/history.parquet")
concatenated_df = pd.concat([df_history_train, df_history_val], ignore_index=True)
concatenated_df = concatenated_df[["user_id", "article_id_fixed"]]
result_df = concatenated_df.groupby('user_id', as_index=False).agg({'article_id_fixed': aggregate_hobbies})
# print(len(df_history_train[df_history_train["user_id"] == 14241]["article_id_fixed"][1]))
# print(len(df_history_val[df_history_val["user_id"] == 14241]["article_id_fixed"][0]))
# print(len(result_df[result_df["user_id"] == 14241]["article_id_fixed"][33]))
result_df

,user_id,article_id_fixed
0,10068,"[9735753, 9739065, 9667501, 9757508, 9759671, ..."
1,10200,"[9764325, 9763923, 9763854, 9423230, 9763854, ..."
2,10201,"[9738557, 9738569, 9738490, 9738528, 9736689, ..."
3,10385,"[9777320, 9777475, 9777529, 9778028, 9778192, ..."
4,10623,"[9739035, 9739065, 9738303, 9739065, 9737266, ..."
...,...,...
18822,2590039,"[9778168, 9778198, 9778168, 9778192, 9778236, ..."
18823,2590054,"[9741288, 9743490, 9741288, 9738659, 9743320, ..."
18824,2590471,"[9770533, 9770491, 9770532, 9770491, 9769433, ..."
18825,2590571,"[9762694, 9764086, 9764049, 9763977, 9763880, ..."


In [220]:
exploded_df = result_df.explode('article_id_fixed')
merged_df = pd.merge(exploded_df, df_articles_train, left_on='article_id_fixed', right_on='article_id')
df3 = merged_df.groupby('user_id').agg({'category': list}).reset_index()
df3

,user_id,category
0,10068,"[457, 414, 498, 414, 414, 414, 414, 118, 118, ..."
1,10200,"[118, 118, 118, 118, 414, 414, 140, 140, 140, ..."
2,10201,"[118, 118, 512, 512, 2975, 140, 142, 457, 118,..."
3,10385,"[118, 118, 512, 2975, 498, 2975, 142, 118, 414..."
4,10623,"[414, 414, 140, 140, 118, 118, 118, 118, 414, ..."
...,...,...
18822,2590039,"[498, 414, 414, 414, 140, 140, 140, 140, 118, ..."
18823,2590054,"[414, 414, 414, 118, 414, 140, 498, 118, 118, ..."
18824,2590471,"[140, 142, 142, 118, 118, 142, 142, 118, 142, ..."
18825,2590571,"[118, 142, 2975, 142, 142, 414, 118, 140, 118,..."


In [224]:
def unique_sorted_categories(categories):
    # Count the frequency of each category
    category_counts = pd.Series(categories).value_counts()
    # Sort categories by frequency
    sorted_categories = category_counts.index.tolist()
    return sorted_categories

df31 = merged_df.groupby('user_id')['category'].apply(unique_sorted_categories).reset_index()
df31


,user_id,category
0,10068,"[414, 118, 457, 140, 142, 498]"
1,10200,"[140, 118, 414, 142, 2975, 512, 498, 565, 457]"
2,10201,"[118, 142, 414, 140, 2975, 512, 498, 457]"
3,10385,"[118, 142, 2975, 512, 498, 414, 457]"
4,10623,"[118, 142, 140, 414, 512, 457, 2975, 529, 2077..."
...,...,...
18822,2590039,"[140, 414, 498, 142, 118]"
18823,2590054,"[414, 118, 140, 498]"
18824,2590471,"[142, 118, 140]"
18825,2590571,"[142, 118, 414, 140, 2975, 457, 498, 512]"


In [306]:
guessed_right = 0
guessed_wrong = 0
guesses = []
for index, row in df_train.iterrows():
    curUser = row["user_id"]
    articles_in_view = row["article_ids_inview"]
    actually_picked = row["article_ids_clicked"] # array
    df_filtered = df_articles_train[df_articles_train['article_id'].isin(articles_in_view)]
    df_filtered = df_filtered.reset_index(drop=True)
    category_list = df31[df31["user_id"] == curUser].iloc[0]["category"]

    df_filtered = df_filtered[df_filtered['category'].isin(category_list)]

    if len(df_filtered) == 0:
        picked = random.choice(row["article_ids_inview"])
        guesses.append([])
    else:

        # Sort df_filtered based on the index of category_list
        df_filtered['category_index'] = df_filtered['category'].apply(lambda x: category_list.index(x))
        df_filtered = df_filtered.sort_values(by='category_index')

        # Select the leftmost article
        picked = df_filtered.iloc[0]["article_id"]
        guesses.append(df_filtered["article_id"].tolist())
    if picked in row["article_ids_clicked"]:
        guessed_right += 1
    else:
        guessed_wrong += 1




print(guessed_right, guessed_wrong)
print(guessed_right/guessed_wrong)
print(guessed_right/(guessed_wrong + guessed_right))

df_answers["most_clicked_categories"] = guesses
df_answers

33064 199823
0.16546643779745074
0.14197443395294715


,impression_id,user_id,article_ids_inview,article_ids_clicked,highest_sentiment_score,most_clicked_categories
0,149474,139836,"[9778623, 9778682, 9778669, 9778657, 9778736, ...",[9778657],"[9778623, 9778728, 9778682, 9778669, 9778657, ...","[9778657, 9778669, 9778736, 9778623, 9778728]"
1,150528,143471,"[9778718, 9778728, 9778745, 9778669, 9778657, ...",[9778623],"[9778718, 9778623, 9778728, 9778682, 9778669, ...","[9778657, 9778669, 9778736, 9778623, 9778728, ..."
2,153068,151570,"[9778657, 9778669, 9772866, 9776259, 9756397, ...",[9778669],"[9776259, 9778682, 9778669, 9756397, 9778657, ...","[9778657, 9778669, 9756397, 9772866, 9776259, ..."
3,153070,151570,"[9020783, 9778444, 9525589, 7213923, 9777397, ...",[9778628],"[9778444, 9430567, 9778718, 9777397, 9525589, ...","[9778628, 9430567, 9525589, 9020783, 9777397, ..."
4,153071,151570,"[9777492, 9774568, 9565836, 9335113, 9771223, ...",[9777492],"[9770218, 9778623, 9777492, 9131971, 9335113, ...","[9777492, 9131971, 9335113, 9565836, 9770218, ..."
...,...,...,...,...,...,...
232882,580099643,2106715,"[9233208, 9771242, 9767697, 9514481, 9771065, ...",[9770886],"[9770886, 9771242, 9771083, 9769306, 9767697, ...","[9770886, 9771128, 9521144, 9540165, 9767697, ..."
232883,580099644,2106715,"[9771065, 9767697, 9770886, 9758882, 9709817, ...",[9769306],"[9770886, 9742735, 9771242, 9709817, 9769917, ...","[9666599, 9709817, 9742735, 9758882, 9767697, ..."
232884,580099645,2106715,"[9771042, 9440508, 9486080, 9770997, 9120051, ...",[9771042],"[9770886, 9767697, 9440508, 9470078, 9770997, ...","[9470078, 9767697, 9770886, 9771042, 9486080, ..."
232885,580100695,2110744,"[9440508, 9142581, 9769917, 9767697, 9514481, ...",[9767697],"[9770886, 9769917, 9767697, 9514481, 9440508, ...","[9767697, 9770886, 9769917, 8422665, 9142581, ..."


In [ ]:
print(guessed_right, guessed_wrong)
print(guessed_right/guessed_wrong)
print(guessed_right/(guessed_wrong + guessed_right))

In [287]:
def complex_formula(hs, ls, cat, ii):
    if ii == 0:
        return hs
    if ii == 1:
        return ls
    if type(cat) is list:
        return hs
    if ii == 2:
        return cat
    if cat == ls | cat == hs:
        return cat
    return hs

for i in range(0,4):
    new_field = []
    for index, row in df_answers.iterrows():
        new_field.append(complex_formula(row['highestsentiment_score'], row['lowest_sentiment_score'], row['most_clicked_category'], i))

    df_answers['after_quick_maths'] = new_field #df_answers.apply(lambda row: complex_formula(row['highestsentiment_score'], row['lowest_sentiment_score'], row['most_clicked_category'], i))
    guessed_right = 0
    guessed_wrong = 0
    for index, row in df_answers.iterrows():
        if row["after_quick_maths"] in row["article_ids_clicked"]:
            guessed_right += 1
        else:
            guessed_wrong += 1

    print(f"strategy {i}")
    print(guessed_right/(guessed_wrong + guessed_right))

strategy 0
0.12948861011988702
strategy 1
0.12657420691853977
strategy 2
0.13323278029160382
strategy 3
0.12948861011988702


In [300]:
df_answers

,impression_id,user_id,article_ids_inview,article_ids_clicked,not_already_clicked,highestsentiment_score,lowest_sentiment_score,most_clicked_category,after_quick_maths,highest_sentiment_score,most_clicked_categories
0,96791,22548,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],"[9783865, 9784591, 9784679, 9784696, 9784710]",9784591,9784710,9784591,9784591,"[9784591, 9783865, 9784679, 9784696, 9784710]","[9784591, 9784679, 9784696, 9784710, 9783865]"
1,96798,22548,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],"[9782884, 9783865, 9782726, 9695098, 9782806, ...",9779370,9784506,9769155,9779370,"[9779370, 9774559, 9783865, 9784648, 9784160, ...","[9769155, 9784648, 9784642, 9784583, 9784506, ..."
2,96801,22548,"[9784648, 7184889, 9784444, 9781785, 9783405, ...",[9784444],"[9784648, 7184889, 9784444, 9781785, 9783405, ...",9782108,9784444,9784444,9782108,"[9782108, 9781785, 9784648, 7184889, 9783405, ...","[9784444, 9784648, 9781520, 9781785, 9781983, ..."
3,96808,22548,"[9784607, 9695098, 9784406, 7184889, 9781983]",[9781983],"[9784607, 9695098, 9784406, 7184889, 9781983]",7184889,9781983,9781983,7184889,"[7184889, 9695098, 9784607, 9784406, 9781983]","[9781983, 9784607, 9784406]"
4,96810,22548,"[9781983, 7184889, 9784642, 9782108, 9781785, ...",[9784642],"[9781983, 7184889, 9784642, 9782108, 9781785, ...",9782108,9781983,9784642,9782108,"[9782108, 9781785, 7184889, 9781520, 9784642, ...","[9784642, 9781520, 9781785, 9781983, 9782108]"
...,...,...,...,...,...,...,...,...,...,...,...
244642,579250707,1529985,"[9782929, 9782919, 9783213, 9783075, 9782746, ...",[9783075],"[9782929, 9782919, 9783213, 9783075, 9782746, ...",9783075,9783213,9782645,9783075,"[9783075, 9782645, 9782919, 9782746, 9782929, ...","[9782645, 9782919, 9783075, 9782746, 9783213, ..."
244643,579251515,1532837,"[9782746, 9782929, 9783720, 9782645, 9783075]",[9783075],"[9782746, 9782929, 9783720, 9782645, 9783075]",9783720,9782929,9782645,9783720,"[9783720, 9783075, 9782645, 9782746, 9782929]","[9782645, 9783720, 9783075, 9782746, 9782929]"
244644,579251519,1532837,"[9783213, 9782645, 9782423, 9783657, 9782746, ...",[9782423],"[9783213, 9782645, 9782423, 9783657, 9782746, ...",9783720,9783657,9782645,9783720,"[9783720, 9783075, 9782645, 9782919, 9782746, ...","[9782645, 9782919, 9783720, 9782423, 9783075, ..."
244645,579253363,1539939,"[9142564, 9783657, 9783404, 9782670, 9783720, ...",[9783720],"[9142564, 9783657, 9783404, 9782670, 9783720, ...",9783720,9782092,9783720,9783720,"[9783720, 9783075, 9783667, 9782645, 9782355, ...","[9783720, 9783643, 9783626, 9782645, 9782919, ..."


In [309]:
toparq = df_answers[["impression_id", "user_id", "article_ids_inview", "article_ids_clicked", "highest_sentiment_score", "most_clicked_categories"]]
toparq.to_parquet("raw_v0_train.parquet")

,impression_id,user_id,article_ids_inview,not_already_clicked,highest_sentiment_score,most_clicked_categories
0,96791,22548,"[9783865, 9784591, 9784679, 9784696, 9784710]","[9783865, 9784591, 9784679, 9784696, 9784710]","[9784591, 9783865, 9784679, 9784696, 9784710]","[9784591, 9784679, 9784696, 9784710, 9783865]"
1,96798,22548,"[9782884, 9783865, 9782726, 9695098, 9782806, ...","[9782884, 9783865, 9782726, 9695098, 9782806, ...","[9779370, 9774559, 9783865, 9784648, 9784160, ...","[9769155, 9784648, 9784642, 9784583, 9784506, ..."
2,96801,22548,"[9784648, 7184889, 9784444, 9781785, 9783405, ...","[9784648, 7184889, 9784444, 9781785, 9783405, ...","[9782108, 9781785, 9784648, 7184889, 9783405, ...","[9784444, 9784648, 9781520, 9781785, 9781983, ..."
3,96808,22548,"[9784607, 9695098, 9784406, 7184889, 9781983]","[9784607, 9695098, 9784406, 7184889, 9781983]","[7184889, 9695098, 9784607, 9784406, 9781983]","[9781983, 9784607, 9784406]"
4,96810,22548,"[9781983, 7184889, 9784642, 9782108, 9781785, ...","[9781983, 7184889, 9784642, 9782108, 9781785, ...","[9782108, 9781785, 7184889, 9781520, 9784642, ...","[9784642, 9781520, 9781785, 9781983, 9782108]"
...,...,...,...,...,...,...
244642,579250707,1529985,"[9782929, 9782919, 9783213, 9783075, 9782746, ...","[9782929, 9782919, 9783213, 9783075, 9782746, ...","[9783075, 9782645, 9782919, 9782746, 9782929, ...","[9782645, 9782919, 9783075, 9782746, 9783213, ..."
244643,579251515,1532837,"[9782746, 9782929, 9783720, 9782645, 9783075]","[9782746, 9782929, 9783720, 9782645, 9783075]","[9783720, 9783075, 9782645, 9782746, 9782929]","[9782645, 9783720, 9783075, 9782746, 9782929]"
244644,579251519,1532837,"[9783213, 9782645, 9782423, 9783657, 9782746, ...","[9783213, 9782645, 9782423, 9783657, 9782746, ...","[9783720, 9783075, 9782645, 9782919, 9782746, ...","[9782645, 9782919, 9783720, 9782423, 9783075, ..."
244645,579253363,1539939,"[9142564, 9783657, 9783404, 9782670, 9783720, ...","[9142564, 9783657, 9783404, 9782670, 9783720, ...","[9783720, 9783075, 9783667, 9782645, 9782355, ...","[9783720, 9783643, 9783626, 9782645, 9782919, ..."
